In [ ]:
!pip install datasets==2.20.0
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install --upgrade pip
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

In [ ]:
import glob

path = "CEREBRAL_PALSY_AUDIO_FILE_PATH"
raw_data_list = sorted(glob.glob(path))

In [ ]:
print(f"file_list : {raw_data_list[:10]}")
print(len(raw_data_list))

In [ ]:
import glob

path = "CEREBRAL_PALSY_TEXT_FILE_PATH"
labeled_data_list = sorted(glob.glob(path))

In [ ]:
print(f"file_list : {labeled_data_list[:10]}")
print(len(labeled_data_list))

In [ ]:
# Drop Utterance Dataset

import pandas as pd
import os

csv_path = '/YOUR_ROOT/fluency_analysis.csv'

df = pd.read_csv(csv_path)
exclude_basenames = df['File_ids'].dropna().apply(lambda x: os.path.splitext(os.path.basename(x))[0]).tolist()

def filter_excluded_files(file_list, exclude_basenames):
    return [
        f for f in file_list
        if os.path.splitext(os.path.basename(f))[0] not in exclude_basenames
    ]

filtered_audio_list = filter_excluded_files(raw_data_list, exclude_basenames)
filtered_text_list = filter_excluded_files(labeled_data_list, exclude_basenames)

print("Audio File:", len(raw_data_list))
print("Filtered Audio File:", len(filtered_audio_list))
print("Text File:", len(labeled_data_list))
print("Filtered Text File:", len(filtered_text_list))

In [ ]:
from tqdm import tqdm
import pandas as pd

transcript_list = []
for labeled_data in tqdm(labeled_data_list):
    with open(labeled_data, 'r', encoding='UTF8') as f:
        line = f.readline()
        transcript_list.append(line)

df = pd.DataFrame(data=transcript_list, columns = ["transcript"])

df["raw_data"] = raw_data_list

In [ ]:
df.isnull().values.sum()

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import Audio
from datasets import load_dataset

In [ ]:
ds = Dataset.from_dict({"audio": [path for path in df["raw_data"]],
                       "transcripts": [transcript for transcript in df["transcript"]]}).cast_column("audio", Audio(sampling_rate=16000))

train_testvalid = ds.train_test_split(test_size=0.1)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
datasets = DatasetDict({
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"]})

datasets.push_to_hub("YOUR_HUGGINGFACE_DATASET_LOCATION")

In [ ]:
from datasets import load_dataset

dataset = load_dataset("YOUR_HUGGINGFACE_DATASET_LOCATION")

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

In [ ]:
dataset

In [ ]:
from datasets import load_dataset, Audio
import torchaudio
import torchaudio.transforms as T
import torch
import random

In [ ]:
# Noise Injection
def add_noise(waveform, noise_level=0.005):
    noise = torch.randn_like(waveform) * noise_level
    return waveform + noise

# Speed Perturbation
def speed_perturb(waveform, orig_sr, factor_range=(0.9, 1.1)):
    factor = random.uniform(*factor_range)
    new_sr = int(orig_sr * factor)
    waveform = torchaudio.functional.resample(waveform, orig_sr, new_sr)
    waveform = torchaudio.functional.resample(waveform, new_sr, orig_sr)
    return waveform

def augment_audio(example):
    waveform = torch.tensor(example["audio"]["array"]).unsqueeze(0)  # [1, T]
    sr = example["audio"]["sampling_rate"]

    if random.random() < 0.5:
        waveform = add_noise(waveform)
    if random.random() < 0.5:
        waveform = speed_perturb(waveform, sr)

    example["audio"]["array"] = waveform.squeeze(0).numpy().astype("float32")
    return example

In [ ]:
from datasets import DatasetDict, Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

def is_valid_audio(example):
    audio_len = len(example["audio"]["array"])
    transcript = example["transcripts"].strip()
    return (
        16_000 <= audio_len <= 480_000 
        and len(transcript) >= 2 
        and not transcript.isspace()
)

dataset = DatasetDict({
    split: dataset[split].filter(is_valid_audio)
    for split in ["train", "valid", "test"]
})

In [ ]:
augmented_train = dataset["train"].map(augment_audio, num_proc=None)

augmented_dataset = DatasetDict({
    "train": augmented_train,
    "valid": dataset["valid"],
    "test": dataset["test"]
})

In [ ]:
from datasets import concatenate_datasets

original_train = dataset["train"]

augmented_train = original_train.map(augment_audio, num_proc=None)

merged_train = concatenate_datasets([original_train, augmented_train])

merged_dataset = DatasetDict({
    "train": merged_train,
    "valid": dataset["valid"],
    "test": dataset["test"]
})

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(batch["transcripts"]).input_ids
    return batch

In [ ]:
processed_dataset = merged_dataset.map(
    prepare_dataset,
    remove_columns=dataset["train"].column_names,
    num_proc=None,
    batched=False
)

In [ ]:
processed_dataset.push_to_hub("YOUR_HUGGINGFACE_PREPROCESS_DATASET_LOCATION")

## Fine-Tuning

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline, WhisperConfig
from datasets import load_dataset

model_id = "openai/whisper-small"

config = WhisperConfig.from_pretrained(model_id)
config.dropout = 0.1
config.attention_dropout = 0.1

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    config=config
)


processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True
)

dataset = load_dataset("YOUR_HUGGINGFACE_PREPROCESS_DATASET_LOCATION")
# If you want to use Provided dataset, use this code
# dataset = load_dataset("yoona-J/ASR_Preprocess_Celebral_Palsy_Dataset_Aug")

In [ ]:
dataset

In [ ]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt", padding=True)

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import locale
print(locale.getpreferredencoding())

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
pip install nlptutti

In [ ]:
from nlptutti.asr_metrics import get_wer, get_cer

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids = [
        [token if token != -100 else tokenizer.pad_token_id for token in sentence]
        for sentence in label_ids
    ]

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    pred_text = " ".join([s.strip() for s in pred_str])
    label_text = " ".join([s.strip() for s in label_str])


    cers, wers = [], []
    for p, l in zip(pred_text, label_text):
        cer_result = get_cer(l, p, rm_punctuation=True)
        wer_result = get_wer(l, p, rm_punctuation=True)
        cers.append(cer_result["cer"])
        wers.append(wer_result["wer"])

    return {"cer": sum(cers)/len(cers) * 100, "wer": sum(wers)/len(wers) * 100}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="YOUR_HUGGINGFACE_MODEL_LOCATION",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate= 3e-5,
    warmup_steps=200,
    max_steps=2828,                    # epoch 14
    lr_scheduler_type="linear",
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=400,
    eval_steps=400,
    logging_steps=50,
    weight_decay = 0.0,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
    push_to_hub=False,
    save_total_limit=2
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor
)

trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "YOUR_HUGGINGFACE_PREPROCESS_DATASET_LOCATION",
    # If you want to use Provided dataset, use this code
    # "dataset_tags": "yoona-J/ASR_Preprocess_Celebral_Palsy_Dataset_Aug",
    "dataset": "YOUR_HUGGINGFACE_PREPROCESS_DATASET_LOCATION",
    "dataset_args": "config: ko, split: valid",
    "language": "ko",
    "model_name": "YOUR_HUGGINGFACE_MODEL_LOCATION",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [ ]:
processor.push_to_hub("YOUR_HUGGINGFACE_MODEL_LOCATION")
tokenizer.push_to_hub("YOUR_HUGGINGFACE_MODEL_LOCATION")